In [1]:
import re
import random
import numpy as np
import shapefile as spf
from shapely.geometry import shape, Point

In [78]:
%config Completer.use_jedi = False

In [44]:
def get_rcoordinates(country, path='World_Countries.shp'):  
    
    def is_match(text):
        match = re.match(f'.*{country}.*', text)
        return match
    
    # Read the shape file that contains list of countries with their unique ID
    country_list = spf.Reader(path)
    
    # get the record that belong to the country we have queried the function
    try:
        country = [c for c in country_list.records() if is_match(c.__str__()) is not None][0] # [0] to extract the only item from the list
    except IndexError:
        return None
    
    # Extract the country ID from the variable country
    country_id = int(re.findall(r'\d+', country.__str__())[0])
    
    # Get the coordinates from the polygon details dictionary
    coordinates_list = country_list.shapeRecords()[country_id].shape.__geo_interface__['coordinates']
    
    coordinates = None
    tol = 100
    count = 0
    while count < tol:
        if len(coordinates_list) == 2 and isinstance(coordinates_list[0], float):
            coordinates = coordinates_list
            break
        else:
            coordinates_list = coordinates_list[0]
        count += 1
    
    return coordinates_list

In [45]:
tmp = get_rcoordinates('Ellala')

In [46]:
tmp

In [47]:
!pip install pandas

In [16]:
import pandas as pd

In [65]:
nodes = pd.read_csv('../data/mmpgroupsfull.csv', header=0)

In [66]:
nodes.head(3)

,group_id,group_name,startyear,endyear,active,complete,description,on_any_map,map_name,_merge
0,1,The Islamic State,2002.0,0.0,1.0,1.0,The Islamic State (IS) also known as the Islam...,1.0,Aleppo,matched (3)
1,3,Mujahideen Army,2004.0,0.0,1.0,1.0,The Mujahideen Army (MA) is a Sunni Iraqi mili...,1.0,Global Al Qaeda,matched (3)
2,5,Islamic Army in Iraq,2003.0,0.0,1.0,1.0,The Islamic Army in Iraq (IAI) is a Sunni Isla...,1.0,Global Al Qaeda,matched (3)


In [70]:
nodes['lat'] = 0
nodes['long'] = 0
for i in range(nodes.shape[0]):
    coordinates = get_rcoordinates(nodes.loc[i, 'map_name'])
    if coordinates is not None:
        nodes.loc[i, 'long'] = coordinates[1]
        nodes.loc[i, 'lat'] = coordinates[0]

In [71]:
nodes.loc[(nodes['long'] == 0) & (nodes['lat'] == 0), 'map_name'].unique()

array(['Aleppo', 'Global Al Qaeda', 'Pakistan -- All', nan,
       'Global Islamic State', 'Northern Ireland', 'North Caucasus',
       'North Africa', 'Kurdistan', 'Global Right-Wing Extremism'],
      dtype=object)

In [72]:
nodes.shape

(385, 12)

In [73]:
get_rcoordinates('Kurdistan')

In [74]:
nodes.loc[nodes['map_name']=='Aleppo', ['long', 'lat']] = get_rcoordinates('Syria')

In [75]:
nodes.loc[nodes['map_name']=='Global Al Qaeda', ['long', 'lat']] = get_rcoordinates('Iraq') # For now

In [76]:
nodes.loc[nodes['map_name']=='Pakistan -- All', ['long', 'lat']] = get_rcoordinates('Pakistan') # For now

In [80]:
nodes.to_csv('../data/new_nodes.csv')